# 实体对齐

In [3]:
include("../src/translatedata.jl")
mkpath("EA_data")

"EA_data"

In [4]:
dict_word_zh2en = Dict((=>).(last.(ILLs), first.(ILLs)))

Dict{SubString{String}, SubString{String}} with 9774 entries:
  "Cambridge_United_Football_Club"     => "Cambridge_United_FC"
  "san_amaro_da_imperatris"            => "Santo_Amaro_da_Imperatriz"
  "Somi_M1931_submachine_gun"          => "Suomi_KP_/_-31"
  "Ramiro_III_(Leon)"                  => "Ramiro_III_of_Leon"
  "Naval_Moralejo"                     => "Navalmoralejo"
  "Abade"                              => "Abadeh"
  "Theodor_Swedberg"                   => "Theodor_Svedberg"
  "Portwater_County_(Kansas)"          => "Pottawatomie_County,_Kansas"
  "FBI_director"                       => "Director_of_the_Federal_Bureau_of_In…
  "Hennepin_County_(Minnesota)"        => "Hennepin_County,_Minnesota"
  "Azov_Sea"                           => "Sea_of_Azov"
  "Chelyabinsk_Region"                 => "Chelyabinsk_Oblast"
  "onturia_de_la_cantera"              => "Hontoria_de_la_Cantera"
  "west_jerez"                         => "Sigeres"
  "Godfrey_Harold_Hardy"               => "GH_Hard

In [9]:
open("EA_data/triples.txt", "w") do io
    for (sub, rel, obj) in triples
        sub = get(dict_word_zh2en, sub, sub)
        obj = get(dict_word_zh2en, sub, sub)
        println(io, sub, '\t', rel, '\t', obj)
    end
end
open("EA_data/train_ques.txt", "w") do io
    for sol in train_ques_sols
        que, ner, rels = sol[1], sol[2], sol[3:end]
        ner = get(dict_word_zh2en, ner, ner)
        println(io, que, '\t', ner, '\t', join(rels, '\t'))
    end
end

open("EA_data/valid_ques.txt", "w") do io
    for (que, ner) in valid_ques_ner
        ner = get(dict_word_zh2en, ner, ner)
        println(io, que, '\t', ner)
    end
end

In [24]:
c = sort_count(last.(wiki_ills))

OrderedDict{String, Int64} with 40887 entries:
  "靚太唔易做_(第… => 23
  "靚太唔易做_(第… => 21
  "大英帝國勳章"  => 7
  "法兰克国王列表… => 7
  "天主教會"      => 6
  "西班牙自治區"  => 6
  "阿斯特里德公主… => 5
  "美国州份"      => 5
  "聖米迦勒及聖喬… => 5
  "哲學博士"      => 4
  "新教"          => 4
  "巴斯勳章"      => 4
  "罗曼诺夫王朝"  => 4
  "奧地利大公國"  => 4
  "汉朝"          => 4
  "万代南梦宫娱乐… => 4
  "自殺攻擊"      => 4
  "几内亚比绍总统… => 4
  "缅甸行政区划"  => 4
  "近世"          => 4
  "越南君主列表"  => 4
  "老挝君主列表"  => 4
  "車輛號牌"      => 4
  "美國郵區編號"  => 4
  "巴基斯坦行政区… => 4
  ⋮               => ⋮

In [23]:
filter(i->i[2] == "靚太唔易做_(第二季)", wiki_ills)

23-element Vector{Tuple{String, String}}:
 ("Desperate_Housewives_(season_2)", "靚太唔易做_(第二季)")
 ("You_Could_Drive_a_Person_Crazy", "靚太唔易做_(第二季)")
 ("You'll_Never_Get_Away_from_Me", "靚太唔易做_(第二季)")
 ("My_Heart_Belongs_to_Daddy_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("They_Asked_Me_Why_I_Believe_in_You", "靚太唔易做_(第二季)")
 ("I_Wish_I_Could_Forget_You", "靚太唔易做_(第二季)")
 ("Color_and_Light", "靚太唔易做_(第二季)")
 ("The_Sun_Won't_Set_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("That's_Good,_That's_Bad_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("Coming_Home_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("One_More_Kiss_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("We're_Gonna_Be_All_Right", "靚太唔易做_(第二季)")
 ("There's_Something_About_a_War", "靚太唔易做_(第二季)")
 ("Silly_People_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("Thank_You_So_Much", "靚太唔易做_(第二季)")
 ("There_Is_No_Other_Way_(Desperate_Housewives)", "靚太唔易做_(第二季)")
 ("Could_I_Leave_You%3F", "靚太唔易做_(第二季)")
 ("Everybody_Says_Don't_(season_two_episode)", "靚太唔易做_(第二季)")
 (

In [21]:
filter(i->i[1]=="ZIP_Code", wiki_ills)

8-element Vector{Tuple{String, String}}:
 ("ZIP_Code", "美國郵區編號")
 ("ZIP_Code", "美國郵政編號")
 ("ZIP_Code", "美国邮政编码")
 ("ZIP_Code", "ZIP_code")
 ("ZIP_Code", "美國郵政編碼")
 ("ZIP_Code", "美国邮区编号")
 ("ZIP_Code", "美國郵遞區號")
 ("ZIP_Code", "美国邮政编号")

In [7]:
function sort_count(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

sort_count (generic function with 1 method)

In [8]:
# 对齐函数
entity_zh2en(ent) = haskey(wiki_ills, ent) ? wiki_ills[ent] : ent
triple_byills(triple::Tuple) = triple_byills(triple...)
triple_byills(sub, rel, obj) = (entity_zh2en(sub), MT_rels[rel], entity_zh2en(obj))

triple_byills (generic function with 2 methods)

In [16]:
# 三元组对齐合并
triples = triple_byills.(vcat(zh_triples, en_triples))
uniquetriples = unique(triples)
length.([triples, uniquetriples])

2-element Vector{Int64}:
 267485
 262564

In [7]:
# 写入文件
open("ILLs/triples.txt", "w") do io
    for triple in uniquetriples
        println(io, join(triple, '\t'))
    end
end

In [17]:
using DataStructures
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in triples
    push!(edges[sub], (rel, obj))
end

In [18]:
subs = unique!(first.(triples))
# 第一跳
jumps = Dict(sub => [length(edges[sub])] for sub in subs)
# 第二跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][1] for (rel, obj) in edges[sub]))
end
# 第三跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][2] for (rel, obj) in edges[sub]))
end

In [19]:
jump_three(sub) = haskey(jumps, sub) ? jumps[sub][3] : 0
jump_two(sub) = haskey(jumps, sub) ? jumps[sub][2] : 0

jump_two (generic function with 1 method)

In [20]:
fail2, fail3 = String[], String[]
for (que, ner) in train_ques_ner
    jump_two(ner) == 0 && push!(fail2, ner)
    jump_three(ner) == 0 && push!(fail3, ner)
end

In [32]:
unique(en_words)

122523-element Vector{String}:
 "Government_of_the_Republic_of_China_in_Guangzhou"
 "Zhang_Xiaoya"
 "Random_Thoughts_(Faye_Wong_album)"
 "Republic_of_China_presidential_election,_1913"
 "French_frigate_Descartes"
 "French_submarine_Archimède"
 "Charles_Simonyi"
 "HSwMS_Thordön"
 "Iraqi_parliamentary_election,_1984"
 "Silulu_A'etonu"
 "Zhou_dynasty_(690–705)"
 "Lakas–CMD"
 "USRC_James_Madison_(1807)"
 ⋮
 "Giulio_Ciccone"
 "Vassiliki_Thanou-Christophilou"
 "Cherbourg-en-Cotentin"
 "Old_Road_F.C."
 "Carin_Strömberg"
 "Catalonia_Government_2016-"
 "Team_BMS_BIRN"
 "List_of_heads_of_government_of_Ethiopia"
 "Ohio_Aviators_(rugby)"
 "Denver_Stampede"
 "2016–17_Championnat_National"
 "Philip_May"

In [34]:
train_ques_sols

1-element Vector{Vector{SubString{String}}}:
 [""]

In [42]:
2000 * 15000

30000000

In [30]:
sort(getindex.(values(jumps), 3), rev=true)

107531-element Vector{Int64}:
 5448
 3924
 3759
 3746
 3466
 2755
 2701
 2622
 2589
 2532
 2416
 2398
 2392
    ⋮
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0

In [23]:
fail3

2556-element Vector{String}:
 "茶裏王"
 "Changes_(advertisement)"
 "Xinjiang_under_Qing_rule"
 "賈斯汀·杜魯多"
 "约翰·肯尼迪"
 "Villa_Farnesina"
 "乌克兰苏维埃社会主义共和国"
 "安昌公主"
 "珉"
 "Pobeda_(watch)"
 "Piaggio_Ciao"
 "Toyoha_mine"
 "Camafuca_diamond_mine"
 ⋮
 "弗朗索瓦二世_(法兰西)"
 "沙姆什·阿达德五世"
 "亨利·利文斯顿"
 "安曼國際體育場"
 "蒂蒂克拉"
 "亨利·利文斯顿"
 "弗朗索瓦二世_(法兰西)"
 "亨利·利文斯顿"
 "克里斯蒂娜·费尔南德斯·德基什内尔"
 "沙姆什·阿达德五世"
 "亨利·利文斯顿"
 "黎棟國"

In [21]:
fail2

1360-element Vector{String}:
 "賈斯汀·杜魯多"
 "约翰·肯尼迪"
 "乌克兰苏维埃社会主义共和国"
 "休斯敦火箭"
 "葡萄牙國家足球隊"
 "丹麥足球協會"
 "土耳其國家足球隊"
 "侠胆雄狮"
 "侠胆雄狮_(2012年电视剧)"
 "肯塔基州副州长"
 "伊格爾·瓦西里耶維奇·庫爾恰托夫"
 "班都斯戰役"
 "阿方索四世_(莱昂)"
 ⋮
 "弗朗索瓦二世_(法兰西)"
 "沙姆什·阿达德五世"
 "亨利·利文斯顿"
 "安曼國際體育場"
 "蒂蒂克拉"
 "亨利·利文斯顿"
 "弗朗索瓦二世_(法兰西)"
 "亨利·利文斯顿"
 "克里斯蒂娜·费尔南德斯·德基什内尔"
 "沙姆什·阿达德五世"
 "亨利·利文斯顿"
 "黎棟國"

In [22]:
# 实体对齐合并
"用 ILLs 翻译实体和关系"
zh2en_byILLs(st::String) = haskey(ILLs_zh_en, st) ? ILLs_zh_en[st] : st
zh2en_byILLs(triple::Tuple) = zh2en_byILLs.(triple)
zh_triples_byILLs = zh2en_byILLs.(zh_triples)

104941-element Vector{Tuple{String, String, String}}:
 ("Sichuan", "capital", "成都市")
 ("Sichuan", "西北", "青海")
 ("Sichuan", "北", "甘肃")
 ("Sichuan", "東北", "陕西")
 ("Sichuan", "西", "西藏")
 ("Sichuan", "東", "重庆")
 ("Sichuan", "南", "云南")
 ("Faye_Wong", "語言", "國語")
 ("Faye_Wong", "語言", "粤語")
 ("Faye_Wong", "語言", "英語")
 ("Faye_Wong", "配偶", "竇唯")
 ("Faye_Wong", "配偶", "李亞鵬")
 ("袁世凯", "monarch", "宣統帝")
 ⋮
 ("古特曼·贝拉", "managerclubs", "沃绍什体育俱乐部")
 ("港區_(東京都)", "自治體", "品川區")
 ("卡伦顿战役", "result", "詹姆斯党")
 ("演播时刻", "network", "英國廣播公司第二台")
 ("亚当·博格丹", "nationalteam", "Hungary_national_under-21_football_team")
 ("江陵之战_(208年)", "combatant", "孙权")
 ("江陵之战_(208年)", "commander", "曹仁")
 ("闲山岛海战", "commander", "九鬼嘉隆")
 ("Kyodo_Television", "majorShareholder", "關西電視台")
 ("稻生之戰", "place", "尾張國")
 ("阿利安薩石油", "league", "哥倫比亞足球甲級聯賽")
 ("長島一向一揆", "partof", "石山合戰")

In [ ]:
"问题简化（去除逗号）"
function refine_que(que)
    ',' ∈ que || return que
    parts = split(que, ',')
    strip(parts[2])[1] == 'a' || return que
    join(parts[[1,3:end...]])
end

In [12]:
"""翻译三元组"""
function write_excel_triple(triples, filename)
    filename = split(filename, '.')[1]
    part1 = @view(triples[1:(5 * 10 ^ 4)])
    part2 = @view(triples[(5 * 10 ^ 4 + 1):end])
    write_xlsx(filename * "_1.xlsx", permutedims(hcat(collect.(part1)...)))
    write_xlsx(filename * "_2.xlsx", permutedims(hcat(collect.(part2)...)))
end

function read_excel_triple(filename)
    filename = split(filename, '.')[1]
    part1 = permutedims(read_xlsx(filename * "_1_MT.xlsx"))
    part2 = permutedims(read_xlsx(filename * "_2_MT.xlsx"))
    triples = NTuple{3, String}[]
    for (sub, rel, obj) in eachcol(hcat(part1, part2))
        push!(triples, (
                replace(strip(sub), ' '=>'_'),
                replace(strip(rel), ' '=>""), 
                replace(strip(obj), ' ' => '_')))
    end
    triples
end

read_excel_triple (generic function with 1 method)

In [13]:



# 处理翻译后的图谱
en_triples_lower = readtriples("translate/triple_en_lower.txt")
zh_triples_lower = readtriples("translate/triple_zh_lower.txt")

# 原始图谱
raw_en_triples = readtriples("raw_triple/triple_en.txt")
raw_zh_triples = readtriples("raw_triple/triple_zh.txt")

# 建立翻译关系
MT_en_raw2new = Dict((=>).(raw_en_triples, en_triples_lower))
MT_zh_raw2new = Dict((=>).(raw_zh_triples, zh_triples_lower))
MT_en_new2raw = Dict((=>).(en_triples_lower, raw_en_triples))
MT_zh_new2raw = Dict((=>).(zh_triples_lower, raw_zh_triples))

ner_raw2new = Dict((=>).(first.(vcat(raw_en_triples, raw_zh_triples)),
        first.(vcat(en_triples_lower, zh_triples_lower))))

ner_new2raw = Dict((=>).(first.(vcat(raw_en_triples, raw_zh_triples)),
        first.(vcat(en_triples_lower, zh_triples_lower))))


LoadError: SystemError: opening file "translate/triples.txt": 没有那个文件或目录